In [ ]:
!kaggle datasets download shantanudhakadd/bank-customer-churn-prediction

Dataset URL: https://www.kaggle.com/datasets/shantanudhakadd/bank-customer-churn-prediction
License(s): other
  0% 0.00/262k [00:00<?, ?B/s]
100% 262k/262k [00:00<00:00, 81.1MB/s]


In [ ]:
!unzip /content/bank-customer-churn-prediction.zip

Archive:  /content/bank-customer-churn-prediction.zip
  inflating: Churn_Modelling.csv     


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
df = pd.read_csv('/content/Churn_Modelling.csv')

In [ ]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [ ]:
df.dtypes

,0
RowNumber,int64
CustomerId,int64
Surname,object
CreditScore,int64
Geography,object
Gender,object
Age,int64
Tenure,int64
Balance,float64
NumOfProducts,int64


In [ ]:
df.isna().sum()

,0
RowNumber,0
CustomerId,0
Surname,0
CreditScore,0
Geography,0
Gender,0
Age,0
Tenure,0
Balance,0
NumOfProducts,0


In [ ]:
df['Exited'].value_counts()

,count
Exited,
0,7963
1,2037


In [ ]:
X = df.drop(['RowNumber', 'CustomerId', 'Surname', 'Exited'], axis=1)
y = df['Exited'].copy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
X_num = X_train.select_dtypes(include = np.number)
X_cat = X_train.select_dtypes(exclude = np.number)
num_features = X_train.select_dtypes(include = np.number).columns
cat_features = X_train.select_dtypes(exclude = np.number).columns

In [ ]:
num_pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder())
])

transformer = ColumnTransformer([
    ('num', num_pipeline, num_features),
    ('cat', cat_pipeline, cat_features)
], remainder='passthrough')

In [ ]:
full_pipeline_rnd = Pipeline([
    ('preprocessing', transformer),
    ('rnd_clf', RandomForestClassifier(random_state=42,
                                       class_weight='balanced'
                                       ))
])

full_pipeline_rnd.fit(X_train, y_train)
full_pipeline_rnd.score(X_test, y_test)

0.8672

In [2]:
# param_grid = {
#     'rnd_clf__n_estimators':[50,100,150,200,500],
#     'rnd_clf__max_depth':[2,5,10,15,50],
#     'rnd_clf__min_samples_split':[2,5,10,15,50],
#     'rnd_clf__min_samples_leaf':[2,5,10,15,50]
# }

# grid = GridSearchCV(full_pipeline_rnd,param_grid = param_grid,cv = 3)
# grid.fit(X_train,y_train)

In [ ]:
grid.best_score_

In [ ]:
grid.best_params_

In [ ]:
full_pipeline_svc = Pipeline([
    ('preprocessing', transformer),
    ('svc', SVC())
])
full_pipeline_svc.fit(X_train, y_train)
full_pipeline_svc.score(X_test, y_test)

0.8572

In [ ]:
full_pipeline_log = Pipeline([
    ('preprocessing', transformer),
    ('log_reg', LogisticRegression())])
full_pipeline_log.fit(X_train, y_train)
full_pipeline_log.score(X_test,y_test)

0.8072

In [ ]:
voting_clf=VotingClassifier(
    estimators= [
        ('lr', LogisticRegression(random_state=42)),
        ('rf', RandomForestClassifier(random_state=42)),
        ('svc', SVC(random_state=42, probability=True))])

full_pipeline_voting = Pipeline([
    ('preprocessing', transformer),
    ('vtg_clf', voting_clf)
])

full_pipeline_voting.fit(X_train, y_train)
full_pipeline_voting.score(X_test,y_test)

0.8584